In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries

In [ ]:
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
!pip install autocorrect
from autocorrect import Speller
from collections import OrderedDict 
from pathlib import Path
import shutil
import spacy 
import re
nltk.download('wordnet')
nltk.download('stopwords')
!pip install contractions
!pip install spacy
!pip install textblob
import contractions
from textblob import TextBlob, Word

In [ ]:
import joblib

Cleaning
1. regex cleaning
2.lowecasing
3.spelling (not good)
4.puctuation stopwords
5.extraction + stemming

In [ ]:
documentsDict = {}
def cleaning(filepath,filename,flag):
  if(flag==1):
    f = open(filepath,'r',errors = 'ignore').read()
    clean_S = contractions.fix(f)
    clean_S=re.sub("([\-]{1,2})", " ",clean_S)
    clean_s=re.sub("[\,]", " ",clean_S)
    clean_S = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", clean_S)  #junk symbols
    # clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S=re.sub("([\.\!]{2})", " ",clean_S)  #removing punctuations
    clean_S = re.sub('\s+', ' ', clean_S)    
    clean_S=re.sub("[^a-zA-Z\s\n]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
    clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S = clean_S.lower()
    documentsDict[filename]=clean_S

  elif(flag==0):
    filepath = contractions.fix(filepath)
    filepath=re.sub("([\-]{1,2})", " ",filepath)
    filepath=re.sub("[0-9]", "",filepath)
    # filepath=re.sub("[\,]", " ",filepath)
    filepath = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", filepath)
    # filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath=re.sub("([\.\!]{2})", " ",filepath)
    filepath = re.sub('\s+', ' ', filepath)
    filepath=re.sub("[^a-zA-Z\s\n]", " ",filepath)
    filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath = filepath.lower()
    return filepath

In [ ]:
files = os.listdir('/content/drive/MyDrive/Sem2/IR/Ass1/stories-20210226T135554Z-001/stories')
for file in files:
       cleaning('/content/drive/MyDrive/Sem2/IR/Ass1/stories-20210226T135554Z-001/stories/'+file,file,1)  

Tokenization & Punctuation Removal

In [ ]:
def gen_token(flag,query):
  tokenizer = RegexpTokenizer(r'\w+')
  if(flag==1):
    for docname in documentsDict:
        documentsDict[docname] = tokenizer.tokenize(str(documentsDict[docname]))

  elif(flag==0):
    query = tokenizer.tokenize(query)
    return query

Removing Stopwords

In [ ]:
def remove_stopword(flag,query):
  stopwordsList = stopwords.words('english')
  if(flag==1):
    for docname in documentsDict:
        for word in documentsDict[docname]:
            if word in stopwordsList:
                documentsDict[docname].remove(word)
  elif(flag==0):
      q=[]
      for word in query:
            if word not in stopwordsList:
              q.append(word)
      query=q.copy()
      return query          

In [ ]:
gen_token(1,"")
remove_stopword(1,"")

Lemmatization using textblob

In [ ]:
def normalise_using_textblob(flag,query):
  if(flag==1):       
    for docname in documentsDict:
        dataList = []
        for word in documentsDict[docname]:
            w = Word(word)
            dataList.append(w.lemmatize())
        documentsDict[docname] = dataList   

  elif(flag==0):
    ans = []
    for word in query:
      w=Word(word)
      ans.append(w.lemmatize())
    return ans        

Unigram Inverted Index Data Structure

In [ ]:
vocabulary = []
file = open('vocab.txt','w')
for docname in documentsDict:
    for word in documentsDict[docname]:
        vocabulary.append(word)

vocabulary = set(vocabulary)
for word in vocabulary:
   file.write(word+" , ")
file.close

In [ ]:
invertedIndex = {}
postfile = open('posting.txt','w') 
for term in vocabulary:
    postingList = []
    postfile.write("\n"+term + " - ")
    for docid in documentsDict:
        if term in documentsDict[docid]:
            postingList.append(docid)
            postfile.write(docid+" , ")
    invertedIndex[term] = postingList  

In [ ]:
#joblib.dump(invertedIndex, '/content/drive/MyDrive/IR/invertedIndex')
invertedIndex = joblib.load('/content/drive/MyDrive/IR/invertedIndex') 
postings=invertedIndex

Input Query and Operator

In [ ]:
def proquery(query,operator):
  queryFinal=[]
  query = cleaning(query," ",0)
  queryFinal = gen_token(0,query)
  queryFinal = remove_stopword(0,queryFinal)
  queryFinal = normalise_using_textblob(0,queryFinal)
  #queryFinal = queryFinal.replace(","," ").split()
  operator = re.sub("[^a-zA-Z\s\n\,]", " ",operator)
  op = operator.split(",")
  op = [x.strip(" ") for x in op]
  op = [x.upper() for x in op]
  return queryFinal,op 

In [ ]:
def partc(queryFinal,op):
  count = 0
  word1=queryFinal[0]
  word2=queryFinal[1]
  docList1=[]
  docList2=[]
  finalList=[]
  for i in range(len(queryFinal)-1):
      if op[i] == 'AND':
        if i == 0 :
          if (word1 in postings.keys()):
           docList1 = postings[word1].copy()
          if (word2 in postings.keys()):
           docList2 = postings[word2].copy()
        else :
          docList1=[]
          docList1 = finalList
          finalList = []
          word2=queryFinal[i+1]
          if (word2 in postings.keys()):
              docList2=[]
              docList2 = postings[word2].copy()
        for item in docList1:
            if item in docList2:
              count +=1
              finalList.append(item)
      if op[i] == 'OR':
        if i == 0 :
          if (word1 in postings.keys()):
           docList1 = postings[word1].copy()
          if (word2 in postings.keys()):
           docList2 = postings[word2].copy()
        else :
          docList1=[]
          docList1 = finalList
          finalList = []
          word2=queryFinal[i+1]
          if (word2 in postings.keys()):
              docList2=[]
              docList2 = postings[word2].copy()
        for item in docList1:
            count += 1
            finalList.append(item)
            if item in docList2:
              docList2.remove(item)
        for item in docList2:
          count += 1
          finalList.append(item)
      if op[i] == 'AND NOT':
        if i == 0 :
          if (word1 in postings.keys()):
           docList1 = postings[word1].copy()
          if (word2 in postings.keys()):
           docList2 = postings[word2].copy()
          for item in docList1:
            count += 1
            finalList.append(item)
            if item in docList2:
              finalList.remove(item)
           
        else :
          docList1=[]
          docList1 = finalList
          finalList = []
          word2=queryFinal[i+1]
          if (word2 in postings.keys()):
              docList2=[]
              docList2 = postings[word2].copy()
          for item in docList1:
            finalList.append(item)
            if item in docList2:
              finalList.remove(item)
              count += 1
      if op[i] == 'OR NOT':
        if i == 0 :
          if (word1 in postings.keys()):
           docList1 = postings[word1].copy()
          for item in postings.keys():
             if item != word2:
               for u in postings[item]:
                  if u not in docList2:
                    docList2.append(u)
        
        else :
          docList1=[]
          docList1 = finalList
          finalList = []

          word2=queryFinal[i+1]
          docList2=[]
          for item in postings.keys():
             if item != word2:
               for u in postings[item]:
                    if u not in docList2:
                       docList2.append(u)
        for item in docList1:
            count += 1
            finalList.append(item)
            if item in docList2:
              docList2.remove(item)
           
        for item in docList2:
          count += 1
          finalList.append(item)
  print("Output - ")
  print("Number of documents matched: ",len(finalList))
  print("No. of comparisons required: ",count)
  print(finalList)

In [ ]:
query = "lion STO1OD 1 thoughtfully a for a moment"
operator = "OR,OR,OR"
queryFinal,op=proquery(query,operator)
partc(queryFinal,op)

Output - 
Number of documents matched:  270
No. of comparisons required:  695
['yukon.txt', 'vgilante.txt', 'veiledl.txt', 'abbey.txt', 'beggars.txt', 'aesop11.txt', 'aesopa10.txt', 'girlclub.txt', 'fgoose.txt', 'holmesbk.txt', 'monkking.txt', 'mouslion.txt', 'lionmosq.txt', 'lionmane.txt', 'mazarin.txt', 'lionwar.txt', 'redragon.txt', 'pussboot.txt', 'silverb.txt', 'blasters.fic', 'quarter.c18', 'archive', 'bookem2', 'fic5', 'grav', 'chik', 'idi.hum', 'korea.s', '13chil.txt', '6napolen.txt', '3gables.txt', 'wlgirl.txt', 'advsayed.txt', 'wisteria.txt', '3student.txt', '5orange.txt', '4moons.txt', 'zombies.txt', '7voysinb.txt', 'ab40thv.txt', 'alad10.txt', 'blh.txt', 'bishop00.txt', 'bruce-p.txt', 'aislesix.txt', 'blackp.txt', 'aquith.txt', 'arctic.txt', 'blind.txt', 'cooldark.txt', 'bulfelis.txt', 'bulphrek.txt', 'bulmrx.txt', 'bureau.txt', 'buldream.txt', 'darkness.txt', 'dskool.txt', 'cmoutmou.txt', 'cybersla.txt', 'bulprint.txt', 'diaryflf.txt', 'bulzork1.txt', 'cardcnt.txt', 'fanta

In [ ]:
query = "telephone,   paved, roads"
operator = "[ OR NOT, AND NOT]"
queryFinal,op=proquery(query,operator)
partc(queryFinal,op)

Output - 
Number of documents matched:  348
No. of comparisons required:  586
['aircon.txt', 'bulzork1.txt', 'excerpt.txt', 'hotline4.txt', 'graymare.txt', 'paink-ws.txt', 'running.txt', 'space.txt', 'telefone.txt', 'sqzply.txt', 'jaynejob.asc', 'ghost', 'szechuan', 'timem.hac', 'progx', 'excerpt.hum', 'corcor.hum', 'korea.s', 'eyeargon.hum', '3wishes.txt', 'bagelman.txt', 'bagel.man', 'dakota.txt', 'nigel.10', '4moons.txt', 'modemhippy.txt', 'startrek.txt', 'sre02.txt', 'sre03.txt', 'deathmrs.d', 'quarter.c3', 'rocket.sf', 'sre_finl.txt', 'quest', 'elite.app', 'fable.txt', 'beast.asc', 'tao3.dos', 'knuckle.txt', 'panama.txt', 'rainda.txt', '18.lws', 'times.fic', 'dan', 's&m_that', 'sre09.txt', 'bulnland.txt', 'horsdonk.txt', 'sanpedr2.txt', 'sre_sei.txt', 'blue', 'bishop00.txt', 'burintrv.92', 'spectacl.poe', 'berternie.txt', 'how.ernie.bert', 'hell4.txt', 'fic3', 'batlslau.txt', 'buldream.txt', 'bulhuntr.txt', 'bulironb.txt', 'cardcnt.txt', 'girlclub.txt', 'tcoa.txt', 'tearglas.txt',

In [ ]:
query = "equality1, sculpted?, plenipotentiary"
operator = "[ OR NOT, AND NOT]"
queryFinal,op=proquery(query,operator)
partc(queryFinal,op)

Output - 
Number of documents matched:  466
No. of comparisons required:  468
['gulliver.txt', 'outcast.dos', 'tao3.dos', 'aesopa10.txt', 'perf', 'enchdup.hum', 'eyeargon.hum', '3wishes.txt', 'bagelman.txt', 'diaryflf.txt', 'bagel.man', 'aesop11.txt', 'dakota.txt', 'darkness.txt', 'long1-3.txt', 'nigel.10', '4moons.txt', 'beggars.txt', 'fgoose.txt', 'hound-b.txt', 'hellmach.txt', 'hitch2.txt', 'hitch3.txt', 'modemhippy.txt', 'startrek.txt', 'sre02.txt', 'sre03.txt', '16.lws', 'deathmrs.d', 'quarter.c3', 'crazy.hum', 'rocket.sf', 'sre_finl.txt', 'quest', 'elite.app', 'fable.txt', 'history5.txt', 'beast.asc', 'timem.hac', 'cooldark.txt', 'cybersla.txt', 'knuckle.txt', 'panama.txt', 'rainda.txt', 'radar_ra.txt', '18.lws', 'times.fic', 'archive', 'dan', 's&m_that', 'cooldark.sto', 'sre09.txt', 'bulnland.txt', 'bureau.txt', 'horsdonk.txt', 'lament.txt', 'fic5', 'sanpedr2.txt', 'sick-kid.txt', 'solitary.txt', 'sre_sei.txt', 'blue', 'bishop00.txt', '6napolen.txt', 'burintrv.92', 'spectacl.poe

In [ ]:
query = "equality,sculpted,plenipotentiary"
operator = "[  AND NOT,OR NOT]"
queryFinal,op=proquery(query,operator)
partc(queryFinal,op)

Output - 
Number of documents matched:  467
No. of comparisons required:  470
['gulliver.txt', 'outcast.dos', 'tao3.dos', 'aesopa10.txt', 'perf', 'enchdup.hum', 'eyeargon.hum', '3wishes.txt', 'bagelman.txt', 'diaryflf.txt', 'bagel.man', 'aesop11.txt', 'dakota.txt', 'darkness.txt', 'long1-3.txt', 'nigel.10', '4moons.txt', 'beggars.txt', 'fgoose.txt', 'hound-b.txt', 'hellmach.txt', 'hitch2.txt', 'hitch3.txt', 'modemhippy.txt', 'startrek.txt', 'sre02.txt', 'sre03.txt', '16.lws', 'deathmrs.d', 'quarter.c3', 'crazy.hum', 'rocket.sf', 'sre_finl.txt', 'quest', 'elite.app', 'fable.txt', 'history5.txt', 'beast.asc', 'robotech', 'timem.hac', 'cooldark.txt', 'cybersla.txt', 'knuckle.txt', 'panama.txt', 'rainda.txt', 'radar_ra.txt', '18.lws', 'times.fic', 'archive', 'dan', 's&m_that', 'cooldark.sto', 'sre09.txt', 'bulnland.txt', 'bureau.txt', 'horsdonk.txt', 'lament.txt', 'fic5', 'sanpedr2.txt', 'sick-kid.txt', 'solitary.txt', 'sre_sei.txt', 'blue', 'bishop00.txt', '6napolen.txt', 'burintrv.92', '

In [ ]:
query = "abandon babble cabby"
operator = "[AnD NOT, OR NOT]"
queryFinal,op=proquery(query,operator)
partc(queryFinal,op)

Output - 
Number of documents matched:  467
No. of comparisons required:  487
['ab40thv.txt', 'blackp.txt', 'hansgrtl.txt', 'hound-b.txt', 'paink-ws.txt', 'sre-dark.txt', 'outcast.dos', 'lil', 'ezoff', 'superg1', 'timem.hac', 'enchdup.hum', 'consumdr.hum', 'goldbug.poe', 'rocket.sf', 'korea.s', 'spectacl.poe', 'domain.poe', 'aesopa10.txt', 'gulliver.txt', 'perf', 'eyeargon.hum', '3wishes.txt', 'bagelman.txt', 'diaryflf.txt', 'bagel.man', 'aesop11.txt', 'dakota.txt', 'darkness.txt', 'long1-3.txt', 'nigel.10', '4moons.txt', 'beggars.txt', 'fgoose.txt', 'hellmach.txt', 'hitch2.txt', 'hitch3.txt', 'modemhippy.txt', 'startrek.txt', 'sre02.txt', 'sre03.txt', '16.lws', 'deathmrs.d', 'quarter.c3', 'crazy.hum', 'sre_finl.txt', 'quest', 'elite.app', 'fable.txt', 'history5.txt', 'beast.asc', 'robotech', 'tao3.dos', 'cooldark.txt', 'cybersla.txt', 'knuckle.txt', 'panama.txt', 'rainda.txt', 'radar_ra.txt', '18.lws', 'times.fic', 'archive', 'dan', 's&m_that', 'cooldark.sto', 'sre09.txt', 'bulnland.t

In [ ]:
query = "SUMMER winter cold RAIN pig"
operator = "[AND nOT, OR, AND NOT, OR NOT]"
queryFinal,op=proquery(query,operator)
partc(queryFinal,op)

Output - 
Number of documents matched:  467
No. of comparisons required:  797
['vgilante.txt', 'adler.txt', 'bagelman.txt', 'cybersla.txt', 'bulironb.txt', 'bullove.txt', 'enginer.txt', 'hellmach.txt', 'graymare.txt', 'mazarin.txt', 'keepmodu.txt', 'shoscomb.txt', 'solitary.txt', 'taxnovel.txt', 'burintrv.66', 'burintrv.78', 'nigel.4', 'bagel.man', 'quarter.c2', 'blackrdr', 'bookem2', 'lil', 'inter', 'ghost', 'fic7', 'enc', 'fic4', 'non4', 'altside.hum', 'write', 'enchdup.hum', 'consumdr.hum', 'cameloto.hum', 't_zone.jok', 'korea.s', 'arcadia.sty', 'yukon.txt', '6napolen.txt', '100west.txt', 'wlgirl.txt', '7oldsamr.txt', '6ablemen.txt', 'zombies.txt', 'veiledl.txt', 'abbey.txt', 'alad10.txt', 'blh.txt', 'beautbst.txt', 'batlslau.txt', 'beggars.txt', 'bishop00.txt', 'bruce-p.txt', 'aesopa10.txt', 'berternie.txt', 'arctic.txt', 'blind.txt', 'antcrick.txt', 'cabin.txt', 'bulphrek.txt', 'bulmrx.txt', 'bureau.txt', 'buldream.txt', 'dskool.txt', 'campfire.txt', 'bulhuntr.txt', 'diaryflf.txt'

In [ ]:
N = input("Enter N: ") 
N=int(N) 
q=[]
o=[]
for i in range(N):
   r=input("Input Query:")
   y=input("Input operation sequence:")
   q.append(r)
   o.append(y)
for i in range(N):
  queryFinal,op=proquery(q[i],o[i])
  partc(queryFinal,op)

Enter N: 2
Input Query:lion STOOD thoughtfully a for a moment
Input operation sequence:OR,OR,OR
Input Query:telephone,   paved, roads
Input operation sequence:[ OR NOT, AND NOT]
Output - 
Number of documents matched:  270
No. of comparisons required:  695
['yukon.txt', 'vgilante.txt', 'veiledl.txt', 'abbey.txt', 'beggars.txt', 'aesop11.txt', 'aesopa10.txt', 'girlclub.txt', 'fgoose.txt', 'holmesbk.txt', 'monkking.txt', 'mouslion.txt', 'lionmosq.txt', 'lionmane.txt', 'mazarin.txt', 'lionwar.txt', 'redragon.txt', 'pussboot.txt', 'silverb.txt', 'blasters.fic', 'quarter.c18', 'archive', 'bookem2', 'fic5', 'grav', 'chik', 'idi.hum', 'korea.s', '13chil.txt', '6napolen.txt', '3gables.txt', 'wlgirl.txt', 'advsayed.txt', 'wisteria.txt', '3student.txt', '5orange.txt', '4moons.txt', 'zombies.txt', '7voysinb.txt', 'ab40thv.txt', 'alad10.txt', 'blh.txt', 'bishop00.txt', 'bruce-p.txt', 'aislesix.txt', 'blackp.txt', 'aquith.txt', 'arctic.txt', 'blind.txt', 'cooldark.txt', 'bulfelis.txt', 'bulphrek.txt